In [ ]:
pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.2 MB/s eta 0:00:00


In [ ]:
pip install nltk

In [ ]:
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import demoji
import spacy
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
#Q1
df = pd.read_csv('emails.csv')
print(df.head())
print("\nDescribing the data")
print(df.describe())

                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1

Describing the data
              spam
count  5728.000000
mean      0.238827
std       0.426404
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000


In [ ]:
#Q2
df = df.drop_duplicates()
df.dropna(inplace=True)

def clean_text(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    return text
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].apply(clean_text)
print("Difference between cleaned and raw data (first 5 rows):\n") #to check raw see upper part
print(df.head())

<ipython-input-10-2e0f70a62c09>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


Difference between cleaned and raw data (first 5 rows):

                                                text  spam
0  subject naturally irresistible your corporate ...     1
1  subject the stock trading gunslinger  fanny is...     1
2  subject unbelievable new homes made easy  im w...     1
3  subject 4 color printing special  request addi...     1
4  subject do not have money  get software cds fr...     1


In [ ]:
#Q3
def remove_emojis(text):
    return demoji.replace(text,'')

df['text'] = df['text'].apply(remove_emojis)

# Display the first 5 rows of the cleaned email body
print(df['text'].head())

0    subject naturally irresistible your corporate ...
1    subject the stock trading gunslinger  fanny is...
2    subject unbelievable new homes made easy  im w...
3    subject 4 color printing special  request addi...
4    subject do not have money  get software cds fr...
Name: text, dtype: object


In [ ]:
#Q4
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

nlp = spacy.load("en_core_web_sm")
def apply_stemming_nltk(text):
    words = nltk.word_tokenize(text) #stemming apply using nltk
    stemmed_words = [stemmer.stem(word) for word in words if word.lower() not in stop_words]
    return " ".join(stemmed_words)

def apply_lemmatization_spacy(text):
    doc = nlp(text) #stemming apply using spacy
    lemmatized_words = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(lemmatized_words)

In [ ]:
# Apply stemming using NLTK
df['stemmed_email_body_nltk'] = df['text'].apply(apply_stemming_nltk)

# Apply lemmatization using spaCy
df['lemmatized_email_body_spacy'] = df['text'].apply(apply_lemmatization_spacy)

print("Stemming:")
print(df['stemmed_email_body_nltk'].head())

print("\nLemmatization:")
print(df['lemmatized_email_body_spacy'].head())

Stemming:
0    subject natur irresist corpor ident lt realli ...
1    subject stock trade gunsling fanni merril muzo...
2    subject unbeliev new home made easi im want sh...
3    subject 4 color print special request addit in...
4    subject money get softwar cd softwar compat gr...
Name: stemmed_email_body_nltk, dtype: object

Lemmatization:
0    subject naturally irresistible corporate ident...
1    subject stock trading gunslinger   fanny merri...
2    subject unbelievable new home easy   m want   ...
3    subject 4 color printing special   request add...
4    subject money   software cd    software compat...
Name: lemmatized_email_body_spacy, dtype: object


In [ ]:
#Q5
manual_stopwords = ["the", "and", "to", "of", "a", "in", "for", "on", "with", "is", "it", "this", "that", "was", "as", "at", "by"]
def remove_stopwords_manual(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in manual_stopwords]
    return ' '.join(filtered_words)
def remove_stopwords_nltk(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['email_body_no_manual_stopwords'] = df['text'].apply(remove_stopwords_manual)
df['email_body_no_nltk_stopwords'] = df['text'].apply(remove_stopwords_nltk)

# Print Email Body with Manual Stop Word Removal
print("Email Body with Manual Stop Word Removal:")
print(df['email_body_no_manual_stopwords'].head())

# Print Email Body with NLTK Stop Word Removal
print("\nEmail Body with NLTK Stop Word Removal:")
print(df['email_body_no_nltk_stopwords'].head())



Email Body with Manual Stop Word Removal:
0    subject naturally irresistible your corporate ...
1    subject stock trading gunslinger fanny merrill...
2    subject unbelievable new homes made easy im wa...
3    subject 4 color printing special request addit...
4    subject do not have money get software cds fro...
Name: email_body_no_manual_stopwords, dtype: object

Email Body with NLTK Stop Word Removal:
0    subject naturally irresistible corporate ident...
1    subject stock trading gunslinger fanny merrill...
2    subject unbelievable new homes made easy im wa...
3    subject 4 color printing special request addit...
4    subject money get software cds software compat...
Name: email_body_no_nltk_stopwords, dtype: object


In [ ]:
#Q6
def tokenize_text(text):
    words = word_tokenize(text)
    return words
df['tokenized_email_body'] = df['text'].apply(tokenize_text)

In [ ]:
print("Tokenized Email Body:")
print(df['tokenized_email_body'].head())

Tokenized Email Body:
0    [subject, naturally, irresistible, corporate, ...
1    [subject, stock, trading, gunslinger, fanny, m...
2    [subject, unbelievable, new, homes, made, easy...
3    [subject, 4, color, printing, special, request...
4    [subject, money, get, software, cds, software,...
Name: tokenized_email_body, dtype: object


In [ ]:
#Q7
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df['email_text'] = df['tokenized_email_body'].apply(lambda x: ' '.join(x))
# TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
# TF-IDF vectors
tfidf_vectors = tfidf_vectorizer.fit_transform(df['email_text'])

In [ ]:
#Q8
from sklearn.feature_extraction.text import CountVectorizer
email_text = df['email_body_no_nltk_stopwords']
vectorizer = CountVectorizer()

# Create BoW representations
bow_matrix = vectorizer.fit_transform(email_text)

In [ ]:
print("Original Text:")
print(df['text'].iloc[0])

# Print the TF-IDF representation for the same email
print("\nTF-IDF:")
print(tfidf_vectors[0])

# Print the BoW representation for the same email
print("\nBag of Words (BoW):")
print(bow_matrix[0])

Original Text:
subject naturally irresistible your corporate identity  lt is really hard to recollect a company  the  market is full of suqgestions and the information isoverwhelminq  but a good  catchy logo  stylish statlonery and outstanding website  will make the task much easier   we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader  it isguite ciear that  without good products  effective business organization and practicable aim it  will be hotat nowadays market  but we do promise that your marketing efforts  will become much more effective  here is the list of clear  benefits  creativeness  hand  made  original logos  specially done  to reflect your distinctive company image  convenience  logo and stationery  are provided in all formats  easy  to  use content management system letsyou  change your website content and even its structure  promptness  you  will see logo drafts within three business days  affordability  your  marketing b